# Adventure Works Cycles - Sales opportunities

<div class="alert alert-block alert-warning">
<b>About this project:</b> The goal of this project is to explore the Adventure Works database to identify opportunities for business enhancement. <br> Firstly I'll use pandas to explore the dataset.<br> 
Adventure Works Cycles is a ficticious multinational manufacturing company, created by Microsoft to provide data for study purpose.<br>
</div>

## Project steps
Summary: Load the sales dataset, join with demographic data of the delivery region, calculate the sales opportunities.<br>
Assumption: the total US sales result by category (%) will be used as the baseline to define the percentage of ideal sales mix.
>1) Import libraries required <br>
>2) Import dataset (from local Microsoft SQL Server).<br>
>3) Model the datasets
>4) Data pre-processing
>5) 

In [1]:
#Importing required libraries
import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
# creating connection with my local server
connectiondata = (
    "Driver={SQL Server};"
    "Server=localhost\SQLEXPRESS;"
    "Database=AdventureWorks2019;"
    "Trusted_Connection=yes;")
## note to add when required; UID=login;PWD=senhadousuario)
connection = pyodbc.connect(connectiondata)
cursor = connection.cursor()

In [2]:
query_sales = "SELECT convert(DATE,sh.OrderDate) as SalesDate, PP.Name as 'Product', FORMAT(sd.OrderQty,'N0') as 'Qty', FORMAT(sd.LineTotal,'N2') as 'InvoiceValue' FROM sales.SalesOrderDetail SD INNER JOIN Sales.SalesOrderHeader SH ON SH.SalesOrderID = SD.SalesOrderID INNER JOIN Production.Product PP ON PP.ProductID = SD.ProductID"
cursor.execute(query_sales)

In [3]:
connection = pyodbc.connect(connectiondata)
cursor = connection.cursor()
dfsales = pd.read_sql(query_sales,connection,parse_dates={'SalesDate': {'format': '%Y-%m-%d'}})
result = dfsales.dtypes
print(result)

SalesDate       datetime64[ns]
Product                 object
Qty                     object
InvoiceValue            object
dtype: object


In [4]:
print(dfsales)
dfsales = pd.DataFrame(dfsales)

        SalesDate                  Product Qty InvoiceValue
0      2011-05-31   Mountain-100 Black, 42   1     2.024,99
1      2011-05-31   Mountain-100 Black, 44   3     6.074,98
2      2011-05-31   Mountain-100 Black, 48   1     2.024,99
3      2011-05-31  Mountain-100 Silver, 38   1     2.039,99
4      2011-05-31  Mountain-100 Silver, 42   1     2.039,99
...           ...                      ...  ..          ...
121312 2014-06-30    Fender Set - Mountain   1        21,98
121313 2014-06-30             AWC Logo Cap   1         8,99
121314 2014-06-30    Fender Set - Mountain   1        21,98
121315 2014-06-30   All-Purpose Bike Stand   1       159,00
121316 2014-06-30             AWC Logo Cap   1         8,99

[121317 rows x 4 columns]


In [5]:
print(dfsales)
print (type(dfsales))

        SalesDate                  Product Qty InvoiceValue
0      2011-05-31   Mountain-100 Black, 42   1     2.024,99
1      2011-05-31   Mountain-100 Black, 44   3     6.074,98
2      2011-05-31   Mountain-100 Black, 48   1     2.024,99
3      2011-05-31  Mountain-100 Silver, 38   1     2.039,99
4      2011-05-31  Mountain-100 Silver, 42   1     2.039,99
...           ...                      ...  ..          ...
121312 2014-06-30    Fender Set - Mountain   1        21,98
121313 2014-06-30             AWC Logo Cap   1         8,99
121314 2014-06-30    Fender Set - Mountain   1        21,98
121315 2014-06-30   All-Purpose Bike Stand   1       159,00
121316 2014-06-30             AWC Logo Cap   1         8,99

[121317 rows x 4 columns]
<class 'pandas.core.frame.DataFrame'>


In [ ]:
query_customer = "select SC.PersonID, PP.FirstName, PP.LastName 
FROM Person.Person PP, sales.Customer SC
WHERE SC.rowguid = PP.rowguid"